In [1]:
from pynq import Overlay
import pynq.lib.dma
import numpy as np
from pynq import Xlnk

In [2]:
import numpy as np
def gen_pattern():
    n_input = 512
    n_output = 10
    Bx,Bw = 7,6
    mask_x = 0x7F
    mask_w = 0x3F
    w = np.random.randint(-2**(Bw-1),2**(Bw-1),size=(n_input,n_output))
    x = np.random.randint(0,2**Bx,size=n_input)
    packets = []
    for i in range(n_input):
        p = 0
        if i % 4 == 0:
            p |= (0b11 if i == 0 else 0b01) << 30
            p |= (x[i+0] & mask_x) << 21
            p |= (x[i+1] & mask_x) << 14
            p |= (x[i+2] & mask_x) << 7
            p |= (x[i+3] & mask_x)
            packets.append(p)
        w0 = 0
        w0 |= (w[i][0] & mask_w) << 24
        w0 |= (w[i][1] & mask_w) << 18
        w0 |= (w[i][2] & mask_w) << 12
        w0 |= (w[i][3] & mask_w) << 6
        w0 |= (w[i][4] & mask_w)
        packets.append(w0)
        w1 = 0
        w1 |= (w[i][5] & mask_w) << 24
        w1 |= (w[i][6] & mask_w) << 18
        w1 |= (w[i][7] & mask_w) << 12
        w1 |= (w[i][8] & mask_w) << 6
        w1 |= (w[i][9] & mask_w)
        packets.append(w1)
    return np.array(packets), w, x

In [3]:
overlay = Overlay('linear.bit')
dma = overlay.axi_dma_0
overlay.reset()

In [4]:
samples, w, x = gen_pattern()
solution = np.dot(w.T, x)
print(samples, solution)

[3425005275  516466031  223442268 ..., 1061858518  472591046   21628366] [-24791  39795 -22249 -33176 -54873  -7431 -44507  78349  42898 -22404]


In [5]:
%%time

xlnk = Xlnk()
out_buffer = xlnk.cma_array(shape=(10,), dtype=np.int32)
in_buffer = xlnk.cma_array(shape=(1152,), dtype=np.int32)
np.copyto(in_buffer, samples)

dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)

CPU times: user 16.3 ms, sys: 0 ns, total: 16.3 ms
Wall time: 15.9 ms


In [6]:
print(out_buffer)

[-24791  39795 -22249 -33176 -54873  -7431 -44507  78349  42898 -22404]
